In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
from pysal.lib import weights
from splot.libpysal import plot_spatial_weights
from esda.moran import Moran, Moran_Local
from splot.esda import moran_scatterplot, plot_local_autocorrelation, lisa_cluster
import matplotlib.pyplot as plt
import folium
import matplotlib.colors as colors

import warnings
warnings.filterwarnings("ignore")

In [ ]:
censo_es = gpd.GeoDataFrame.from_file('basemunicipal.geojson')
censo_es['População'] = censo_es['População'].str.replace(r'\D', '')
censo_es['População'] = censo_es['População'].astype(int)
#Obtendo o valor do PIB real, uma vez que o valor no dataset original se encontra em milhoes
censo_es['PIB2020'] = censo_es['PIB2020'] *1000000
censo_es.head()

In [ ]:
# Plotando choropleth map da coluna Populacao
censo_es.explore(
    tiles='cartodbpositron',
    column='População',
    height='60%',
    width='60%',
    scheme='naturalbreaks',
    cmap='Oranges',
    style_kwds={
        'stroke': True,
        'edgecolor': 'k',
        'linewidth': 0.03,
        'fillOpacity': 1
        },
    legend_kwds={
        'colorbar': False,
        'fmt': '{:.0f}'
        }
    )

In [ ]:
# Plotting choropleth map da coluna PIB2020
censo_es.explore(
    tiles='cartodbpositron',
    column='PIB2020',
    height='60%',
    width='60%',
    scheme='naturalbreaks',
    cmap='Oranges',
    style_kwds={
        'stroke': True,
        'edgecolor': 'k',
        'linewidth': 0.03,
        'fillOpacity': 1
        },
    legend_kwds={
        'colorbar': False,
        'fmt': '{:.0f}'
        }
    )

In [ ]:
# Construindo spatial weights object
w = weights.contiguity.Queen.from_dataframe(censo_es)

# Plotando spatial weights
plot_spatial_weights(w, censo_es);

In [ ]:
# Transformando weights em binario (Se 1 = is neighbor, 0 = not neighbor)
w.transform = "B"

# Showing neighbors indexes
w.neighbors

In [ ]:
# Showing weights
w.weights

In [ ]:
# Calculando Spatial Lag
censo_es['w_pop'] = weights.lag_spatial(w, censo_es['População'])
censo_es['w_PIB'] = weights.lag_spatial(w, censo_es['PIB2020'])


censo_es[['geocodigo', 'População', 'w_pop', 'PIB2020','w_PIB']].head()

In [ ]:
# Calculando Spatial Lag
censo_es.loc[w.neighbors[0], ['geocodigo', 'População', 'w_pop']]['População'].sum()


In [ ]:
# Variable of interest (populacao)
y_pop_count = censo_es['População']

# Calculando Moran I statistic
moran = Moran(y_pop_count, w)

# Moran's I statistic
moran.I


In [ ]:
moran.p_sim

In [ ]:
# Plotando Moran I scatter plot
moran_scatterplot(moran);

In [ ]:
# Variable of interest (PIB2020)
y_total_monthly_income = censo_es['PIB2020']

# Calculating Moran's I statistic
moran = Moran(y_total_monthly_income, w)

# Moran's I statistic
moran.I




In [ ]:
moran.p_sim

In [ ]:
moran_scatterplot(moran);

In [ ]:
# Local Moran's I
pop_count_local_moran = Moran_Local(y_pop_count, w)

# Plotting Local Moran's I scatterplot of pop_count
fig, ax = moran_scatterplot(pop_count_local_moran, p=0.05);

plt.text(1.95, 1, 'HH', fontsize=25)
plt.text(1.95, -1.0, 'HL', fontsize=25)
plt.text(-1.5, 1, 'LH', fontsize=25)
plt.text(-1.5, -1, 'LL', fontsize=25)
plt.show()

In [ ]:
# creating column with local_moran classification
censo_es['pop_count_local_moran'] = pop_count_local_moran.q


# Dict to map local moran's classification codes
local_moran_classification = {1: 'HH', 2: 'LH', 3: 'LL', 4: 'HL'}

# Mapping local moran's classification codes
censo_es['pop_count_local_moran'] = censo_es['pop_count_local_moran'].map(local_moran_classification)


# p-value for each observation/neighbor pair
censo_es['pop_count_local_moran_p_sim'] = pop_count_local_moran.p_sim


# If p-value > 0.05 it is not statistical significant
censo_es['pop_count_local_moran'] = np.where(censo_es['pop_count_local_moran_p_sim'] > 0.05, 'ns', censo_es['pop_count_local_moran'])

censo_es.head()

In [ ]:
# Plotting Local Moran's I classification map of pop_count column
censo_es.explore(
    tiles='cartodbpositron',
    column='pop_count_local_moran',
    height='70%',
    width='70%',
    cmap=[
    '#D7191C', # Red
    '#FDAE61', # Orange
    '#ABD9E9', # Light Blue
    '#2C7BB6', # Blue
    '#D3D3D3'  # Grey
    ],
    style_kwds={
        'stroke': True,
        'edgecolor': 'k',
        'linewidth': 0.03,
        'fillOpacity': 1
        
        },
    )

In [ ]:
# Local Moran's I
total_monthly_income_local_moran = Moran_Local(y_total_monthly_income, w)

# Plotting Local Moran's I scatterplot of pop_count
fig, ax = moran_scatterplot(total_monthly_income_local_moran, p=0.05);

plt.text(1.95, 1, 'HH', fontsize=25)
plt.text(1.95, -1.0, 'HL', fontsize=25)
plt.text(-1.5, 1, 'LH', fontsize=25)
plt.text(-1.5, -1, 'LL', fontsize=25)
plt.show()

In [ ]:
# creating column with local_moran classification
censo_es['total_monthly_income_local_moran'] = total_monthly_income_local_moran.q

# Dict to map local moran's classification codes
local_moran_classification = {1: 'HH', 2: 'LH', 3: 'LL', 4: 'HL'}

# Mapping local moran's classification codes
censo_es['total_monthly_income_local_moran'] = censo_es['total_monthly_income_local_moran'].map(local_moran_classification)

# p-value for each observation/neighbor pair
censo_es['total_monthly_income_local_moran_p_sim'] = total_monthly_income_local_moran.p_sim

# If p-value > 0.05 it is not statistical significant
censo_es['total_monthly_income_local_moran'] = np.where(censo_es['total_monthly_income_local_moran_p_sim'] > 0.05, 'ns', censo_es['total_monthly_income_local_moran'])

# Plotting Local Moran's I classification map of pop_count column
censo_es.explore(
    tiles='cartodbpositron',
    column='total_monthly_income_local_moran',
    height='70%',
    width='70%',
    cmap=[
    '#D7191C', # Red
    '#FDAE61', # Orange
    '#ABD9E9', # Light Blue
    '#2C7BB6', # Blue
    '#D3D3D3'  # Grey
    ],
    style_kwds={
        'stroke': True,
        'edgecolor': 'k',
        'linewidth': 0.03,
        'fillOpacity': 1
        
        },
    )